In [19]:
import numpy as np
import pandas as pd
import xlrd
import openpyxl

import os
folder_name = '成品仓库进出明细'
if not os.path.exists(folder_name):
    os.makedirs(folder_name)

import sys
month=' ('+str(sys.argv[1])+'月)'
    
file_name = '成品仓库进出明细'
input_filename = file_name +'.xls'
output_filename = folder_name + '/'+ file_name+ '_添加类别.xlsx'

df = pd.read_excel(input_filename,sheet_name = 0)
df.columns.values


array(['单据', 'Unnamed: 1', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4', '属性',
       'Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8', 'Unnamed: 9',
       'Unnamed: 10', 'Unnamed: 11', 'Unnamed: 12', 'Unnamed: 13',
       'Unnamed: 14', 'Unnamed: 15', 'Unnamed: 16', '入库统计', 'Unnamed: 18',
       '出库统计', 'Unnamed: 20'], dtype=object)

In [20]:
#清除 "入库统计"
tag_names = list(df.iloc[0])
col_names = list(df.columns.values)

idx_entering_warehouse = col_names.index('入库统计')
str_entering_warehouse_1= col_names[idx_entering_warehouse]
str_entering_warehouse_2 = col_names[idx_entering_warehouse +1]

print(str_entering_warehouse_1)
print(str_entering_warehouse_2)

df.drop([str_entering_warehouse_1,str_entering_warehouse_2], axis=1, inplace = True)


入库统计
Unnamed: 18


In [21]:
#重命名"类别名称"所在列 的标签
tag_names = list(df.iloc[0])
col_names = list(df.columns.values)

idx_typename = tag_names.index('类别名称')
str_typename = col_names[idx_typename]
print(str_typename,'\n')

df.rename(index=str, columns={str_typename: 'type_name'},inplace=True)


str_typesize = col_names[idx_typename + 1]
print(str_typesize, '\n')

df.rename(index=str, columns={str_typesize: 'pd_size'}, inplace = True)

# 添加一列，命名为 type
print(df.columns.values,'\n')
df.insert(idx_typename,'type',df.type_name)
df.columns.values

# drop last line for summary information
df.drop(df.index[len(df)-1], inplace=True)


Unnamed: 8 

Unnamed: 9 

['单据' 'Unnamed: 1' 'Unnamed: 2' 'Unnamed: 3' 'Unnamed: 4' '属性'
 'Unnamed: 6' 'Unnamed: 7' 'type_name' 'pd_size' 'Unnamed: 10'
 'Unnamed: 11' 'Unnamed: 12' 'Unnamed: 13' 'Unnamed: 14' 'Unnamed: 15'
 'Unnamed: 16' '出库统计' 'Unnamed: 20'] 



In [22]:
df.type[df.type_name == '类别名称'] = '类别'

df.type[(df.type_name =='HDF') | (df.type_name == 'HDG') | (df.type_name == 'HDJ') | (df.type == 'HDT') ] = '地砖'
df.type[df.type_name =='HTF'] = '地砖'
df.type[df.type_name == 'XF'] = '地砖'

df.type[(df.type_name =='HNF') | (df.type_name == 'HNG') | (df.type_name == 'HNJ') | (df.type == 'HNT') ] = '瓷片'
df.type[df.type_name == 'PG'] = '瓷片'
df.type[(df.type_name == 'W') | (df.type_name == 'WM')] = '瓷片'
df.type[(df.type_name == 'XG') | (df.type_name == 'XT') | (df.type_name == 'XW')] = '瓷片'

df.type[df.type_name =='HM'] = '木纹砖'
df.type[df.type_name == 'HW'] = '微晶'
df.type[df.type_name == 'HY'] = '全抛釉'
df.type[df.type_name == 'H3S'] = '色晶砖'
df.type[df.type_name == 'HS'] = '大理石'
df.type[df.type_name == 'HNW'] = '镜面砖'
df.type[(df.type_name == 'P') | (df.type_name == 'T')] = '通体砖'
df.type[df.type_name == '浮雕'] = '浮雕'
df.type[df.type_name == '文具'] = '五金仓'
df.type[df.type_name == '木托'] = '木托'
df.type[df.type_name == '捆砖带'] = '捆砖带'
df.type[df.type_name == '换包装'] = '换包装'
df.type[df.type_name == '花片'] = '花片'
df.type[df.type_name == '欧驰'] = '欧驰'
df.type[df.type_name == '腰线'] = '腰线'
df.type[df.type_name == '纸箱'] = '纸箱'
df.type[df.type_name == '瓷砖胶'] = '5KG'
df.type[df.type_name == '圣陶居'] = '圣陶居'
df.type[df.type_name == '展美'] = '展美'

print(df.groupby('type').type.agg('count'))

type
五金仓      18
其它费用      4
圣陶居       1
地砖       59
捆砖带       1
换托费       2
搬运        9
木托        3
浮雕        7
瓷片      448
类别        1
腰线        6
花片        9
运费        3
镜面砖       1
Name: type, dtype: int64


In [23]:
#modified output
output_df = df.copy(deep=True)
output_df.columns = output_df.iloc[0,:]
output_df.drop(['0'], axis=0,inplace = True)
output_df.set_index('编号',inplace = True)

writer = pd.ExcelWriter(output_filename) #creates instance of an excel workboo
output_df.to_excel(writer,'sheet1') #creates workbook 
writer.save() #saves workbook to file in python file directory

In [24]:
## regroup data
df2 = output_df.copy(deep=True)

# fill nan with ‘空白’
df2['销售类型'] = df2['销售类型'].fillna('未注明')

# fill nan for some classification term 
df2['仓库名称'] = df2['仓库名称'].fillna('未注明')
df2['品牌'] = df2['品牌'].fillna('未注明')

print(df2['销售类型'].unique())
print( df2.columns.values )

['未注明' '正常销售' '折价销售' '免费费用' '免费样板' '物资自用' '销售费用' '收费样板' '免费广告']
['日期' '类型' '销售类型' '往来单位' '仓库名称' '产品编号' '产品名称' '类别' '类别名称' '规格' '庄数' '色号'
 '等级' '品牌' '包装' '单位' '单价' '数量(片)' '金额']


In [25]:
# 销售成本 : '正常销售 + 折价销售 + 收费样板 + 销售调价

#sales = df2[ (df2['销售类型'] == '正常销售') | (df2['销售类型'] == '折价销售') | (df2['销售类型'] == '收费样板') | (df2['销售类型'] == '销售调价') ]
sales = df2[ (df2['销售类型'] != '挂账样板') & 
             (df2['销售类型'] != '挂账广告') & 
             (df2['销售类型'] != '免费广告') & 
             (df2['销售类型'] != '免费样板') ]
             #(df2['销售类型'] != '免费费用') ]


# 挂账样板

model = df2[ df2['销售类型'] == '挂账样板' ]

# 挂账广告

adv = df2[ df2['销售类型'] == '挂账广告']

# 免费广告

free_adv = df2[ df2['销售类型'] == '免费广告']

# 免费样板

free_model = df2[ df2['销售类型'] == '免费样板']

# 免费费用

#free_expenses = df2[ df2['销售类型'] == '免费费用']


In [26]:
def handle_worksheet_format(writer, group, title):
    # handle the spreadsheet format
    workbook = writer.book
    worksheet = writer.sheets['sheet1']      
    # set data format
    for idx, col in enumerate(group):  # loop through all columns
        series = group[col]
        if isinstance(series[0],str): char_ratio = 2 
        else: char_ratio = 1 
        max_len = max((
                        series.astype(str).map(len).max() * char_ratio ,  # len of largest item
                        len( str(series.name) * 2)  # len of column name/header
                    )) + 2  # adding a little extra space

        data_fmt = workbook.add_format({'num_format': '0.00',
                                        'align': 'right'})    
        worksheet.set_column(idx+1, idx+1, max_len, data_fmt)  # set column width

    # set index format
    series = group.index
    max_len_idx = max((
            series.astype(str).map(len).max() * 2,  # len of largest item
            len(str(series.name)* 2)  # len of column name/header
            ))+2
    index_fmt = workbook.add_format()
    index_fmt.set_align('right')
    worksheet.set_column(0,0, max_len_idx, index_fmt)

    # set title 
    header_format = workbook.add_format({'bold': True,
                                     'text_wrap': True,
                                     'align': 'center',
                                     'valign': 'vcenter',
                                     'fg_color': '#D7E4BC', 
                                     'border': 1})      
    worksheet_cols = group.shape[1]
    range_str = 'A1:'+chr(ord('A')+worksheet_cols)+'1'
    worksheet.merge_range(range_str,title,header_format)    

In [27]:
def summary_product_category(key, df_temp, out_path=None, title=None):
    if out_path is None:
        output_filename = folder_name +'/summary_'+key+'.xlsx'
        output_filename2 = folder_name +'/往来单位_'+key+'.xlsx'            
    else:
        output_filename = out_path + '/summary_'+key+'.xlsx'
        output_filename2 = out_path +'/往来单位_'+key+'.xlsx'
    
    if title is None:
        title = key+month       

    group = pd.concat([ df_temp.groupby('类别')['数量(片)'].agg(np.sum),
                        df_temp.groupby('类别')['金额'].agg(np.sum)],
                       axis = 1).sort_values(by='金额',ascending = False)

    group.loc[group.shape[0]] = group.sum()
    group.rename(index = {group.shape[0]-1: '总计'}, inplace = True)
    group = group.round(2)

    writer = pd.ExcelWriter(output_filename) #creates instance of an excel workboo
    group.to_excel(writer,startrow = 1, sheet_name= 'sheet1') #creates workbook

    handle_worksheet_format(writer, group, title)

    writer.save() #saves workbook to file in python file directory
    writer.close()
    
    ########################
    # 往来单位
    ########################
    group2 = (df_temp.groupby([df_temp['往来单位'].fillna('空白')])
                    .agg({'金额':'sum'})
                    .reset_index()
                    .set_index('往来单位')
                    .sort_values(by='金额',ascending = False)
             )

    group2.loc[group2.shape[0]] = group2.sum()
    group2.rename(index = {group2.shape[0]-1: '总计'}, inplace = True)
    group2 = group2.round(2)

    writer = pd.ExcelWriter(output_filename2) #creates instance of an excel workboo
    group2.to_excel(writer,startrow = 1, sheet_name = 'sheet1') #creates workbook 

    # handle the spreadsheet format
    
    title = key + ' 往来单位' + month
    handle_worksheet_format(writer, group2, title)

    writer.save() #saves workbook to file in python file directory
    writer.close()
    
    return



In [28]:
key = '销售成本'
summary_product_category(key, sales)

#key ='挂账样板'
#summary_product_category(key, model)

key = '免费样板'
summary_product_category(key, free_model)

key = '免费广告'
summary_product_category(key, free_adv)

#key = '免费费用'
#summary_product_category(key, free_expenses)


In [29]:
# 不同销售类型的金额汇总
def summary_by_sales_type(df2, out_path=None, title=None,term=None):
    if out_path is None:
        output_filename = folder_name + '/不同销售类型金额汇总.xlsx'
    else:
        if term is None:
            output_filename = out_path + '/不同销售类型金额汇总.xlsx'
        else:
            output_filename = out_path + '/不同销售类型金额汇总_'+term+'.xlsx'

    if title is None:
        title = '销售类型金额统计'+month
   
    group = (df2.groupby('销售类型')['金额']
               .agg(np.sum)
               .to_frame()
               .sort_values(by='金额',ascending = False)
            )
    
    # add some additional sub-category
    drop_list = ['免费样板', '免费广告', '挂账样板', '挂账广告']
    sales_cost_index=  group.index
    for x in drop_list:
        if x in sales_cost_index:
            group.loc['统计：'+x] = group.loc[x]
            sales_cost_index = sales_cost_index.drop(x)

    sub_category={'统计：销售成本' : sales_cost_index}

    if sub_category:
        for key, values in sub_category.items():
            group.loc[key] = [0] * group.shape[1]
            for product in values:
                if product in group.index:
                    group.loc[key] = [sum(x) for x in zip(group.loc[key], group.loc[product])]

    #save to 2 digits precision float
    group = group.round(2)
    
    #write to excel file
    writer = pd.ExcelWriter(output_filename) #creates instance of an excel workboo
    group.to_excel(writer,startrow = 1, sheet_name= 'sheet1') #creates workbook

    # handle the spreadsheet format
    handle_worksheet_format(writer, group, title)

    writer.save() #saves workbook to file in python file directory
    writer.close()
    

In [30]:
# 不同销售类型的金额汇总
summary_by_sales_type(df2)

# 按 仓库/品牌 归类

In [31]:
# testing code

df_temp=sales

term_list = df_temp['仓库名称'].unique()
print(term_list)
df_term_list = [df_temp[df_temp['仓库名称']==term] for term in term_list]
df_term_list = [df_temp] + df_term_list

data_list=  [pd.concat([ x.groupby('类别')['数量(片)'].agg(np.sum),
                        x.groupby('类别')['金额'].agg(np.sum)],
                       axis = 1
                      ).sort_values(by='金额',ascending = False)
            for x in df_term_list]

data_list[0].columns = [['所有仓库','所有仓库'], data_list[0].columns]
for i in range(len(term_list)):
    data_list[i+1].columns = [ [term_list[i],term_list[i]], data_list[i+1].columns]        
    
grouped = pd.concat(data_list,axis=1,sort=False).fillna(0.00)
summation = grouped.sum()
summation.name = '总计'
grouped = grouped.append(summation)
grouped

['恒希仓' '广西仓' '南庄仓' '佛山仓' '五金(新)' '样板仓' '佛山2仓']


所有仓库                  恒希仓                  广西仓                南庄仓  \
          数量(片)         金额     数量(片)         金额     数量(片)         金额   数量(片)   
瓷片     90888.00  379435.34  26173.00  101747.40  63849.00  273538.94   866.0   
地砖     16325.00   35522.75   4635.00    9965.25  11594.00   25341.50    90.0   
木托       318.00   22120.00    290.00   20300.00     28.00    1820.00     0.0   
运费       106.02    8208.92     86.56    6885.64     19.46    1323.28     0.0   
捆砖带      438.00    6570.00    438.00    6570.00      0.00       0.00     0.0   
圣陶居      207.00    4347.00      0.00       0.00      0.00       0.00     0.0   
搬运      8982.00    2174.54   8014.00    2046.20      0.00       0.00   484.0   
浮雕       173.00    1710.69      0.00       0.00      0.00       0.00     0.0   
花片        31.00     527.00      0.00       0.00      0.00       0.00     0.0   
腰线       114.00     391.93      0.00       0.00      0.00       0.00     0.0   
其它费用    1700.00     287.80      0.00       0.00   1698.00     169.80     0.0   
换托费     1698.00     152.82      0.00       0.00   1698.00     152.82     0.0   
镜面砖       15.00     111.00     15.00     111.00      0.00       0.00     0.0   
五金仓       52.00      32.96      0.00       0.00      0.00       0.00     0.0   
总计    121047.02  461592.75  39651.56  147625.49  78886.46  302346.34  1440.0   

                 佛山仓          五金(新)          样板仓       佛山2仓       
           金额  数量(片)       金额 数量(片)     金额 数量(片)   金额 数量(片)   金额  
瓷片    4149.00    0.0     0.00   0.0   0.00   0.0  0.0   0.0  0.0  
地砖     216.00    0.0     0.00   0.0   0.00   6.0  0.0   0.0  0.0  
木托       0.00    0.0     0.00   0.0   0.00   0.0  0.0   0.0  0.0  
运费       0.00    0.0     0.00   0.0   0.00   0.0  0.0   0.0  0.0  
捆砖带      0.00    0.0     0.00   0.0   0.00   0.0  0.0   0.0  0.0  
圣陶居      0.00  207.0  4347.00   0.0   0.00   0.0  0.0   0.0  0.0  
搬运     113.46  484.0    14.88   0.0   0.00   0.0  0.0   0.0  0.0  
浮雕       0.00  173.0  1710.69   0.0   0.00   0.0  0.0   0.0  0.0  
花片       0.00   31.0   527.00   0.0   0.00   0.0  0.0   0.0  0.0  
腰线       0.00  101.0   391.93   0.0   0.00   0.0  0.0  13.0  0.0  
其它费用     0.00    2.0   118.00   0.0   0.00   0.0  0.0   0.0  0.0  
换托费      0.00    0.0     0.00   0.0   0.00   0.0  0.0   0.0  0.0  
镜面砖      0.00    0.0     0.00   0.0   0.00   0.0  0.0   0.0  0.0  
五金仓      0.00    0.0     0.00  52.0  32.96   0.0  0.0   0.0  0.0  
总计    4478.46  998.0  7109.50  52.0  32.96   6.0  0.0  13.0  0.0

In [14]:
def product_classify_by_terms(term, term_string, key, df_temp):
    term_list = df_temp[term_string].unique()
    print(key,':',term_list)
    
    for name in term_list:
        df_item = df_temp[df_temp[term_string]==name]        
        out_path =  term+ '/'+ name + '/' + folder_name
        if not os.path.exists(out_path):
            os.makedirs(out_path)    

        name_str = key + '_' + name
        title= key+ ':' + name + month
        summary_product_category(name_str, df_item, out_path, title)

        
def summary_sales_types_classify_by_terms(term, term_string, df_temp):
    term_list = df_temp[term_string].unique()
    
    for name in term_list:
        df_item = df_temp[df_temp[term_string]==name]
        out_path =  term+ '/金额统计_'+folder_name
        if not os.path.exists(out_path):
            os.makedirs(out_path) 
            
        title = '金额统计:'+name+ month
        summary_by_sales_type(df2, out_path, title, name)     

In [49]:
def handle_worksheet_format(writer, df_temp, title):
    workbook = writer.book
    worksheet = writer.sheets['sheet1']    
    
    for idx, col in enumerate(df_temp):  # loop through all columns
        series = df_temp[col]
        max_len = max(( series.astype(str).map(len).max() ,  # len of largest item
                        max( [len(str(x)) for x in series.name])*2  # len of column name/header
                      )) + 2  # adding a little extra space

        data_fmt = workbook.add_format({'num_format': '0.00',
                                        'align': 'right'})    
        worksheet.set_column(idx+1, idx+1, max_len, data_fmt)  # set column width

    # set index format
    series = df_temp.index
    max_len_idx = max((
            series.astype(str).map(len).max() * 2,  # len of largest item
            len(str(series.name))  # len of column name/header
            ))+2
    index_fmt = workbook.add_format()
    index_fmt.set_align('right')
    worksheet.set_column(0,0, max_len_idx, index_fmt)
    
    # set title 
    header_format = workbook.add_format({'bold': True,
                                     'text_wrap': True,
                                     'align': 'center',
                                     'valign': 'vcenter',
                                     'fg_color': '#D7E4BC', 
                                     'border': 1})
    worksheet_cols = df_temp.shape[1]+1
    if (worksheet_cols <=25):
        range_str = 'A1:'+chr(ord('A')+worksheet_cols-1)+'1'
    else:
        last_col_str = chr(ord('A')+ worksheet_cols // 26 - 1) + chr(ord('A')+ (worksheet_cols-1) % 26) + '1'
        range_str = 'A1:'+last_col_str    
    
    worksheet.merge_range(range_str,title,header_format) 
    

def summary_product_classify_by_terms(term, term_string, key, df_temp, out_path, title=None):
    if not os.path.exists(out_path):
        os.makedirs(out_path)
    output_filename = out_path + '/summary_'+key+'_'+term+'分类.xlsx'      
    if title is None:
        title=key+month
    
    term_list = df_temp[term_string].unique()
    if ('未注明' in term_list):
        term_list= term_list.tolist()
        term_list.remove('未注明')
        term_list.append('未注明')
        
    df_term_list = [df_temp[df_temp[term_string]==name] for name in term_list]
    df_term_list = [df_temp] + df_term_list

    data_list=  [pd.concat([ x.groupby('类别')['数量(片)'].agg(np.sum),
                            x.groupby('类别')['金额'].agg(np.sum)],
                           axis = 1
                          ).sort_values(by='金额',ascending = False)
                for x in df_term_list]
    
    data_list[0].columns = [['所有'+term,'所有'+term], data_list[0].columns]
    for i in range(len(term_list)):
        data_list[i+1].columns = [ [term_list[i],term_list[i]], data_list[i+1].columns]        

    grouped = pd.concat(data_list,axis=1,sort=False).fillna(0.00)
    summation = grouped.sum()
    summation.name = '总计'
    grouped = grouped.append(summation)
    
    # round to precison of 2 digits
    grouped = grouped.round(2)
    
    # write to Excel file
    writer = pd.ExcelWriter(output_filename) #creates instance of an excel workbook
    grouped.to_excel(writer,startrow = 1, sheet_name= 'sheet1') #creates workbook    
    
    
    # handle the spreadsheet format
    handle_worksheet_format(writer, grouped, title)       

    writer.save() #saves workbook to file in python file directory
    writer.close()
    

In [16]:
term ='仓库'
term_string = '仓库名称'
term_list = df2[term_string].unique()
print('所有'+term+'名称:',term_list)

out_path =  term+ '/' + folder_name 
def summary_product_by_factory_and_category(key, df_temp, out_path, title=None):
    summary_product_classify_by_terms(term, term_string, key, df_temp, out_path, title)

def product_by_factory(key, df_temp):
    product_classify_by_terms(term, term_string, key, df_temp)
    
    
key = '销售成本'
product_by_factory(key, sales)
summary_product_by_factory_and_category(key, sales, out_path, key)

#key ='挂账样板'
#product_by_factory(key, model)
#summary_product_by_factory_and_category(key, model, out_path, title)

key = '免费样板'
product_by_factory(key, free_model)
summary_product_by_factory_and_category(key, free_model, out_path, key)

key = '免费广告'
product_by_factory(key, free_adv)
summary_product_by_factory_and_category(key, free_adv, out_path, key)

#key = '免费费用'
#product_by_factory(key, free_expenses)
#summary_product_by_factory_and_category(key, free_expenses, out_path, key)


def summary_by_sales_type_and_factory(df_temp):
    summary_sales_types_classify_by_terms(term, term_string, df_temp)

summary_by_sales_type_and_factory(df_temp)

所有仓库名称: ['恒希仓' '广西仓' '南庄仓' '佛山仓' '样板仓' '五金(新)' '佛山2仓']
销售成本 : ['恒希仓' '广西仓' '南庄仓' '佛山仓' '五金(新)' '样板仓' '佛山2仓']
免费样板 : ['恒希仓' '南庄仓' '样板仓' '五金(新)' '广西仓']
免费广告 : ['五金(新)']


In [17]:
term ='品牌'
term_string = '品牌'
term_list = df2[term_string].unique()
print('所有'+term+'名称:',term_list)

out_path =  term+ '/' + folder_name 
def summary_product_by_brand_and_category(key, df_temp, out_path, title=None):
    summary_product_classify_by_terms(term, term_string, key, df_temp, out_path, title)

def product_by_brand(key, df_temp):
    product_classify_by_terms(term, term_string, key, df_temp)
    
key = '销售成本'
product_by_brand(key, sales)
summary_product_by_brand_and_category(key, sales, out_path)

#key ='挂账样板'
#product_by_brand(key, model)
#summary_product_by_brand_and_category(key, model, out_path)

key = '免费样板'
product_by_brand(key, free_model)
summary_product_by_brand_and_category(key, free_model, out_path)

key = '免费广告'
product_by_brand(key, free_adv)
summary_product_by_brand_and_category(key, free_adv, out_path)

#key = '免费费用'
#product_by_brand(key, free_expenses)
#summary_product_by_brand_and_category(key, free_expenses, out_path)


def summary_by_sales_type_and_brand(df_temp):
    summary_sales_types_classify_by_terms(term, term_string, df_temp)

summary_by_sales_type_and_brand(df_temp)

所有品牌名称: ['星晴' '维万家' '恒希' '无' '佛山' '恒希样板' '出口' '广西厂']
销售成本 : ['星晴' '维万家' '恒希' '无' '佛山' '恒希样板' '出口' '广西厂']
免费样板 : ['星晴' '维万家' '恒希样板' '恒希']
免费广告 : ['恒希']


In [18]:
print('success')

success
